In [9]:
import os
import platform
import numpy as np
from scipy.io import wavfile
import librosa

dir_separator = {"Windows": "\\", "Linux": "/"}
system = platform.system()
slash = dir_separator[system]
path = os.getcwd()

sr = 8000 
# Reading data from "dev" folder
dev_path = path + slash + "dev"
dev_data = []
dev_labels = []

# Extracting data from recording (filename = "id_label.wav")
for file in os.listdir(dev_path):
    file_label = file[-5:-4]
    dev_data.append(librosa.load(dev_path + slash + file, sr = sr)[0])
    dev_labels.append(int(file_label))

# Reading data from "eval" folder
eval_path = path + slash + "eval"
eval_data = []


eval_files = sorted(os.listdir(eval_path), key = lambda x: int(x[:-4]))
for file in eval_files: 
    eval_data.append(librosa.load(eval_path + slash + file, sr = sr)[0])  

In [10]:
# Feature extracion using librosa

def extract_features_from_audio_data(audio_data, sr):
    
    chroma_stft = librosa.feature.chroma_stft(y=audio_data, sr=sr)
    rms = librosa.feature.rms(y=audio_data)
    spec_cent = librosa.feature.spectral_centroid(y=audio_data, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=audio_data, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=audio_data, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(audio_data)
    mfcc = librosa.feature.mfcc(y=audio_data, sr=sr)
    mfcc_means = [np.mean(e) for e in mfcc]
    
    features = [np.mean(chroma_stft), np.std(chroma_stft), np.mean(rms), np.std(rms), np.mean(spec_cent), np.std(spec_cent),
                np.mean(spec_bw), np.std(spec_bw), np.mean(rolloff), np.std(rolloff), np.mean(zcr), np.std(zcr)] + mfcc_means

    return features

X_train = np.array([extract_features_from_audio_data(data,sr) for data in dev_data])
y_train = dev_labels  

c:\users\fratt\appdata\local\programs\python\python37\lib\site-packages\librosa\core\pitch.py:146: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


In [187]:
# trying to find the best classifier
# random_forest

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

params = {
    "n_estimators": [1200,1400, 1600, 1800, 2000],
    "max_depth": [5, 8, 15, 25, 50],
    "max_features": ['auto', 'sqrt'],
    "min_samples_split": [2, 5, 10, 15],
    "min_samples_leaf": [2, 5, 10]
}

rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = params, n_iter = 200, cv = 5, random_state=42, scoring='f1_macro', n_jobs = -1)
rf_random.fit(X_train, y_train)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [188]:
best_rf = rf_random.best_estimator_.fit(X_train,y_train)
best_f1_macro = rf_random.best_score_
print(rf_random.best_params_)
print(best_f1_macro)

{'n_estimators': 1800, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 25}
0.9566428952672126


In [12]:
X_test = np.array([extract_features_from_audio_data(data,sr) for data in eval_data])
y_test = best_rf.predict(X_test)

In [13]:
import csv

with open("predictions.csv", mode='w', encoding='utf-8', newline='') as f:
    f_writer = csv.writer(f, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    f_writer.writerow(["Id", "Predicted"])
    for i in range(len(eval_data)):
        f_writer.writerow([i, y_test[i]])